In [1]:
import numpy as np
from coverage import *

import geopandas as gpd
import pandas as pd
import branca
import folium

In [2]:
! mkdir tmp

mkdir: tmp: File exists


In [3]:
num_days = 2
tles = gen_sats(sat_nos=[39084,49260])

# Generate the instrument from basic specs
# times = gen_times(start_yr=2021, start_mo=12, start_day=10, days=num_days, step_min=1) # Adjusted TIRS for 1 min FOVs
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=4000, mm=True) 

times = gen_times(start_yr=2021, start_mo=12, start_day=10, days=num_days, step_min=0.22) # Actual TIRS
inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True) 

# Select cell size for coverage map
xcell_size = ycell_size = 1

Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2022-06-14 04:43:48 UTC>]
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-13 11:24:57 UTC>]
Propogation time: 
 2021-12-10 00:00:00+00:00 
to 
 2021-12-11 23:59:48+00:00


In [4]:
## Batch FOV generation over N satellites
gdfs = []
for tle in tles:
    sat = tle[0]
    poly_df = forecast_fovs(sat, times, inst)
    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

## Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
poly_df['color'] = poly_df['id'].apply(cmap)

plot_df = poly_df[poly_df["lonspan"] < 20].copy()

plot_df.to_file("./tmp/fovs.geojson")
plot_df

,geometry,satellite,id,time,lonspan,color
0,"POLYGON ((144.72361 -49.04234, 142.23681 -48.6...",LANDSAT 8,139084,2021-12-10 00:00:00 UTC,3.198539,#ff0000ff
1,"POLYGON ((144.42215 -49.82363, 141.89678 -49.3...",LANDSAT 8,139084,2021-12-10 00:00:13 UTC,3.257353,#ff0000ff
2,"POLYGON ((144.11290 -50.60424, 141.54742 -50.1...",LANDSAT 8,139084,2021-12-10 00:00:26 UTC,3.318937,#ff0000ff
3,"POLYGON ((143.79538 -51.38416, 141.18815 -50.9...",LANDSAT 8,139084,2021-12-10 00:00:40 UTC,3.383458,#ff0000ff
4,"POLYGON ((143.46907 -52.16335, 140.81839 -51.7...",LANDSAT 8,139084,2021-12-10 00:00:53 UTC,3.451104,#ff0000ff
...,...,...,...,...,...,...
25953,"POLYGON ((100.34082 -81.92779, 96.07727 -80.34...",LANDSAT 9,149260,2021-12-11 23:09:38 UTC,14.150890,#0000ffff
25954,"POLYGON ((94.95486 -82.22708, 91.46401 -80.595...",LANDSAT 9,149260,2021-12-11 23:09:52 UTC,13.768012,#0000ffff
25955,"POLYGON ((89.19789 -82.45515, 86.63529 -80.783...",LANDSAT 9,149260,2021-12-11 23:10:05 UTC,13.104259,#0000ffff
25956,"POLYGON ((83.15017 -82.60542, 81.64401 -80.906...",LANDSAT 9,149260,2021-12-11 23:10:18 UTC,13.113608,#0000ffff


In [5]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

## Filter by AOI - should pass to forecast function instead?
# aoi =  world[world.name == "Brazil"].geometry
aoi =  world[world.continent == "South America"].geometry
# aoi = world[world.name == "United States of America"].geometry

xmin, ymin, xmax, ymax= aoi.total_bounds
plot_df = plot_df.cx[xmin: xmax, ymin: ymax]

In [6]:
## Coverage data analysis for single satellite/ batch of satellites

# 1) Create a grid of equally spaced points
grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)

# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.to_file('./tmp/n_visits.geojson')
grid.n_visits.describe()

count    944.000000
mean       2.381356
std        0.830689
min        1.000000
25%        2.000000
50%        2.000000
75%        2.000000
max        6.000000
Name: n_visits, dtype: float64

In [7]:
## Plotting FOVs

## Make a folium map
m = plot_df.explore(color="color", tooltip=["satellite", "time"])

## View or save
m.save("./tmp/fov_output.html")

In [8]:
## Plotting Revisit Map

## Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

## Create colormap and apply to img
## TODO: Make this our geoTIFF item for STAC catalog
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m.save("./tmp/coverage_output.html")